In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, f1_score, hamming_loss
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from imblearn.over_sampling import RandomOverSampler

In [25]:
df = pd.read_csv('finalmovie.csv')

In [26]:
df.head()

,title,new_genre,corpus,genre_length,action,adventure,animation,biography,comedy,crime,...,horror,music,musical,mystery,romance,sci-fi,sport,thriller,war,western
0,Avatar: The Way of Water,"Action,Adventure,Fantasy",jake sulli life newfound famili form extrasola...,3,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Menu,"Horror,Thriller",young coupl travel remot island eat exclus res...,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,Babylon,"Comedy,Drama,History",tale outsiz ambit outrag excess trace rise fal...,3,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Everything Everywhere All at Once,"Action,Adventure,Comedy",middleag chines immigr swept insan adventur al...,3,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,M3gan,"Horror,Sci-Fi,Thriller",robot engin toy compani build lifelik doll beg...,3,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [27]:
# df = df.drop(['title','new_genre','corpus','genre_length'],axis=1)

In [28]:
# df.iloc[:,4:]

In [29]:
# df1 = df.iloc[:,:3]

In [30]:
# df1.head()

In [31]:
# genres = [genre.split(',') for genre in df['new_genre']]

In [32]:
# from sklearn.preprocessing import MultiLabelBinarizer
# import pickle

In [33]:
# binarizer = MultiLabelBinarizer()
# binarizer.fit(genres)

In [34]:
# with open('binarizer.pkl', 'wb') as f:
#     pickle.dump(binarizer, f)

In [35]:
df.head()

,title,new_genre,corpus,genre_length,action,adventure,animation,biography,comedy,crime,...,horror,music,musical,mystery,romance,sci-fi,sport,thriller,war,western
0,Avatar: The Way of Water,"Action,Adventure,Fantasy",jake sulli life newfound famili form extrasola...,3,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Menu,"Horror,Thriller",young coupl travel remot island eat exclus res...,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,Babylon,"Comedy,Drama,History",tale outsiz ambit outrag excess trace rise fal...,3,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Everything Everywhere All at Once,"Action,Adventure,Comedy",middleag chines immigr swept insan adventur al...,3,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,M3gan,"Horror,Sci-Fi,Thriller",robot engin toy compani build lifelik doll beg...,3,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [36]:
genre_columns = df.drop(['title','new_genre','corpus','genre_length'],axis=1)

In [37]:
y = genre_columns

In [38]:
y.columns

Index(['action', 'adventure', 'animation', 'biography', 'comedy', 'crime',
       'drama', 'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western'],
      dtype='object')

In [39]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np

# Tokenize the text
df['tokens'] = df['corpus'].apply(word_tokenize)

# Create a list of lists of tokens
documents = df['tokens'].tolist()


# Train the Word2Vec model
model = Word2Vec(documents, vector_size=100, window=5, min_count=1, workers=4)



# Encode the text data using Word2Vec
X = np.zeros((len(documents), model.vector_size))
for i, tokens in enumerate(documents):
    X[i] = np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0)

# Create a new DataFrame with the Word2Vec embeddings
embedding_cols = [f'embedding_{i}' for i in range(model.vector_size)]
embedding_df = pd.DataFrame(X, columns=embedding_cols)

# Concatenate the embedding DataFrame with the original DataFrame
df = pd.concat([df, embedding_df], axis=1)

In [40]:
X = df[embedding_cols]

In [41]:
y = genre_columns

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
print(X_train.shape)
print(y_train.shape)

(61285, 100)
(61285, 21)


## Random Forest Classifier

In [53]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
hamming = hamming_loss(y_test, y_pred)

print(f"Random Forest Classifier accuracy: {accuracy:.2f}")
print(f"Random Forest Classifier precision: {precision:.2f}")
print(f"Random Forest Classifier f1-score: {f1:.2f}")
print(f"Random Forest Classifier Hamming loss: {hamming:.2f}")

Random Forest Classifier accuracy: 0.75
Random Forest Classifier precision: 0.97
Random Forest Classifier f1-score: 0.85
Random Forest Classifier Hamming loss: 0.03


## Decision Tree Classifier

In [92]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
hamming = hamming_loss(y_test, y_pred)

print(f"Decision Tree Classifier accuracy: {accuracy:.2f}")
print(f"Decision Tree Classifier precision: {precision:.2f}")
print(f"Decision Tree Classifier f1-score: {f1:.2f}")
print(f"Decision Tree Classifier Hamming loss: {hamming:.2f}")

Decision Tree Classifier accuracy: 0.73
Decision Tree Classifier precision: 0.81
Decision Tree Classifier f1-score: 0.81
Decision Tree Classifier Hamming loss: 0.04


## Saving the word2vec model and RCF as pickle

In [90]:
# # Save the Word2Vec model
# model.save("word2vec.model")

In [91]:
# import pickle

# # Save the Random Forest model
# with open("random_forest_model.pkl", "wb") as f:
#     pickle.dump(rfc, f)

In [92]:
# import pickle

# with open('genre_columns.pkl', 'wb') as f:
#     pickle.dump(genre_columns, f)


In [99]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pickle

# Load the trained Word2Vec model
model = Word2Vec.load("word2vec.model")

# Load the trained Random Forest model
rfc = pickle.load(open("random_forest_model.pkl", "rb"))

# Get user input
user_input = input("Enter a movie description: ")

# Tokenize user input
tokens = word_tokenize(user_input)

# Encode user input using Word2Vec
X = np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0).reshape(1, -1)


# Predict the genre using the Random Forest model
predicted_genre = rfc.predict(X)

# Convert predicted genre to original labels using binarizer's inverse_transform method
predicted_labels = binarizer.inverse_transform(predicted_genre)

# # Predict the genre using the Random Forest model
# # genre = rfc.predict(X)[0]
# genre = rfc.predict(X)
# genre_name = rfc.classes_[genre][0]

print(f"The predicted genre is {predicted_labels}.")

Enter a movie description:  family is haunted by demon


The predicted genre is [('Horror',)].


/Users/subashanannair/tensorflow-test/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [103]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pickle

# Load the trained Word2Vec model
model = Word2Vec.load("word2vec.model")

# Load the trained Random Forest model
rfc = pickle.load(open("random_forest_model.pkl", "rb"))

# Load the binarizer
binarizer = pickle.load(open("binarizer.pkl", "rb"))

# Get user input
user_input = input("Enter a movie description: ")

# Tokenize user input
tokens = word_tokenize(user_input)

# Encode user input using Word2Vec
X = np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0).reshape(1, -1)

# Predict the genre using the Random Forest model
predicted_genre = rfc.predict(X)

# Convert predicted genre to original labels using binarizer's inverse_transform method
predicted_labels = binarizer.inverse_transform(predicted_genre)

print(f"The predicted genre is {predicted_labels[0]}.")

Enter a movie description:  family members are having fun while they fall in love


The predicted genre is ('Comedy', 'Drama', 'Romance').


/Users/subashanannair/tensorflow-test/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


# Final app

In [104]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pickle

# Load the trained Word2Vec model
model = Word2Vec.load("word2vec.model")

# Load the trained Random Forest model
rfc = pickle.load(open("random_forest_model.pkl", "rb"))

# Load the binarizer
binarizer = pickle.load(open("binarizer.pkl", "rb"))

# Get user input
user_input = input("Enter a movie description: ")

# Tokenize user input
tokens = word_tokenize(user_input)

# Encode user input using Word2Vec
X = np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0).reshape(1, -1)

# Predict the genre using the Random Forest model
predicted_genre = rfc.predict(X)

# Convert predicted genre to original labels using binarizer's inverse_transform method
predicted_labels = binarizer.inverse_transform(predicted_genre)

# Extract the predicted genres as a string without parentheses
predicted_genre_str = ", ".join([label.replace("(", "").replace(")", "").replace("'", "") for label in predicted_labels[0]])

print(f"The predicted genre is {predicted_genre_str}.")

Enter a movie description:  family members are having fun while they fall in love


The predicted genre is Comedy, Drama, Romance.


/Users/subashanannair/tensorflow-test/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
